In [1]:
#pip install pygame

In [2]:
import pygame
from sys import exit
import random

import numpy as np

pygame 2.3.0 (SDL 2.24.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class Entity:
    Eid = 0;
    def __init__(self):
        Entity.Eid+=1;
        self.id = Entity.Eid
        self.componenets = {}
        
    def addComponenet(self, name, value):
        self.componenets.update({name:value})
        
    def removeComponenet(self, name):
        del self.componenets[name]

In [4]:
#Systems:

def userInputSystem(entities):
    newEntities = []
 
    events =  pygame.event.get().copy()
    for event in events:
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()
                exit()

    for e in entities:
        for event in events:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    if e.componenets.get('jump') is not None:  
                        e.componenets['jump'] = True
        newEntities.append(e)

    return newEntities

def rendering(width,height):
    pygame.init()
    
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption('FlapPyNet')  

    def renderingSystem(entities):
        #Clear Background
        screen.fill((225,235,255))
        for e in entities:
            #Draw Birds
            if e.componenets.get('bird_x') is not None:
                if e.componenets.get('alive') == True:
                    if(e.componenets.get('color') == 0): 
                        pygame.draw.circle(screen, (255, 0, 0), (e.componenets.get('bird_x'), e.componenets.get('bird_y')), 20)
                    if(e.componenets.get('color') == 1): 
                        pygame.draw.circle(screen, (0, 255, 0), (e.componenets.get('bird_x'), e.componenets.get('bird_y')), 20)
                    if(e.componenets.get('color') == 2): 
                        pygame.draw.circle(screen, (0, 0, 255), (e.componenets.get('bird_x'), e.componenets.get('bird_y')), 20)
                    if(e.componenets.get('color') == 3): 
                        pygame.draw.circle(screen, (255, 255, 0), (e.componenets.get('bird_x'), e.componenets.get('bird_y')), 20)
            #Draw Pipes
            if e.componenets.get('pipe_x') is not None:
                pygame.draw.rect(screen, (0, 255, 0), 
                                 pygame.Rect(e.componenets.get('pipe_x'), e.componenets.get('pipe_y')+75,
                                             50, 500 ))
                pygame.draw.rect(screen, (0, 255, 0), 
                                 pygame.Rect(e.componenets.get('pipe_x'), e.componenets.get('pipe_y')-325-250,
                                             50, 500 ))
        pygame.display.update()
        return(entities)
    return renderingSystem

def physicsSystem(entities):
    newEntities = []
    for e in entities:
        #Speeds:
        if e.componenets.get('vel') is not None:        
            e.componenets['vel'] = e.componenets['vel'] +1;
            if e.componenets.get('bird_y') is not None:
                if e.componenets['vel'] < 50:
                    e.componenets['bird_y'] = e.componenets['bird_y'] + e.componenets['vel'];

        if e.componenets.get('pipe_x') is not None:        
            e.componenets['pipe_x'] = e.componenets['pipe_x'] - 5;
            if e.componenets['pipe_x'] < -60:
                e.componenets['pipe_x'] = 600
                e.componenets['pipe_y'] = 300 - random.randint(0,150)
                
        newEntities.append(e)
    return newEntities

def collosionSystem(entities):
    newEntities = []
    for b in entities:
        if b.componenets.get('bird_y') is not None:
            for p in entities:
                if p.componenets.get('pipe_x') is not None:
                    if p.componenets.get('pipe_x') < b.componenets.get('bird_x')+10 and p.componenets.get('pipe_x') > b.componenets.get('bird_x')-35:
                        if not(p.componenets.get('pipe_y')+55 > b.componenets.get('bird_y') and p.componenets.get('pipe_y')-55 < b.componenets.get('bird_y')):
                            b.componenets['alive'] = False               
        newEntities.append(b)
    return newEntities

def jumpSystem(entities):
    newEntities = []
    for e in entities:
        #Jump
        if e.componenets.get('jump') is not None:  
            if e.componenets.get('jump') == True:
                e.componenets['vel'] = -15;
                e.componenets['jump'] = False;
        newEntities.append(e)
    return newEntities

def rewardSystem(entities):
    newEntities = []
    for e in entities:
        #Jump
        if e.componenets.get('score') is not None:  
            if e.componenets.get('alive') == True:
                e.componenets['score'] = e.componenets['score']+1      
        newEntities.append(e)
    return entities

def AISystem(entities):
    newEntities = []
    for e in entities:
        #Jump
        if e.componenets.get('Brain') is not None:
            min = 1000
            store = [0, 0]
            for p in entities:
                if p.componenets.get('pipe_x') is not None:
                    if p.componenets.get('pipe_x') > e.componenets.get('bird_x') and p.componenets.get('pipe_x') < min:
                        min = p.componenets.get('pipe_x')
                        store = [p.componenets.get('pipe_x')/600.0, p.componenets.get('pipe_y')/400.0]
            e.componenets['jump'] = e.componenets.get('Brain').forward([e.componenets.get('bird_y')/400.0, e.componenets.get('vel')/10, store[0], store[1]])
        newEntities.append(e)
    
    return newEntities

def resetSysten(entities):
    newEntities = []
    livingAgnets = 0;


    
    for e in entities:
        if e.componenets.get('alive') == True:
            livingAgnets = True
            
    if livingAgnets == 0:
        z = 220;

        population = []
        
        #TODO
        # Sort Birds by score
        for e in entities:
            if e.componenets.get('alive') is not None:
                   population.append(e)   

        population.sort(key=lambda x: x.componenets.get('score'), reverse = True)

        pSize = len(population)

        # 25% remains
        pBest = population[0:int(pSize/4.0)]
        pChildren = []
        pMutants = []
        pNew = []
        # 25% children
        for i in range(int(pSize/4.0)):
            pChildren.append(crossoverGene(
                pBest[random.randint(0, len(pBest)-1)].componenets.get('Brain').getGenome(), 
                pBest[random.randint(0, len(pBest)-1)].componenets.get('Brain').getGenome(), 0.5))
            
        # 25% mutants
        for i in range(int(pSize/4.0)):
            pMutants.append(mutateGene(pBest[random.randint(0, len(pBest)-1)].componenets.get('Brain').getGenome(), 0.05))    

        #25% new agents
        for i in range(int(pSize/4.0)):
             pNew.append(NeuralNetwork(4,32,1).getGenome())

        #Best bird genomes
        pBestT = []
        
        for i in pBest:
            pBestT.append(i.componenets.get('Brain').getGenome())
        pBest = pBestT
        
        brainJar = []

        for i in pBest:
            brainJar.append(i)

        for i in pChildren:
            brainJar.append(i)

        for i in pMutants:
            brainJar.append(i)
            
        for i in pNew:
            brainJar.append(i)

        k = 0
        #Reset birds
        for e in entities:
            #Reset Birds
            if e.componenets.get('alive') is not None:
                e.componenets['bird_y'] = random.randint(100,300)
                e.componenets['alive'] = True
                e.componenets['score']=  0
                e.componenets['jump'] = False
                e.componenets['vel'] = 0
                e.componenets['Brain'].setGenome(brainJar[k])
                e.componenets['color'] = int(k/25.0)
                k = k+1;
            #Resets pipes:
            if e.componenets.get('pipe_x') is not None: 
                z = z + 220
                e.componenets['pipe_x'] = 220+z
                e.componenets['pipe_y'] = 300 - random.randint(0,150)
            #Reset Pipes:
            newEntities.append(e)
        return newEntities
    return entities
        
def time(framerate):
    clock = pygame.time.Clock()
    framerate = framerate;
    
    def timeSystem(entities):
        clock.tick(framerate)
        return(entities)
    
    return timeSystem

In [5]:
def game(entities, systems):
    for s in systems:
        entities = s(entities)

In [6]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def getGenome(self):
        return [self.weights_input_hidden.flatten(), self.weights_hidden_output.flatten()]

    def setGenome(self, genome):
        self.weights_input_hidden = np.reshape(genome[0],(self.input_size, self.hidden_size))
        self.weights_hidden_output = genome[1]
    
    def forward(self, X):
        # Forward pass
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        self.output = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        return self.output[0][0] > 0

In [7]:
def mutateGene(genome, rate):
    getnome3 = []
    for l,i in enumerate(genome):
        genomet = []
        for j, k in enumerate(i):
            if(random.random() > rate):
                genomet.append(2 *random.random()-1)
            else:
                genomet.append(k)
        getnome3.append(np.array(genomet))
    return np.array(np.array(getnome3))
    
def crossoverGene(genome1, genome2, rate):
    getnome3 = []
    for l,i in enumerate(genome1):
        genomet = []
        for j, k in enumerate(i):
            if(random.random() > rate):
                genomet.append(genome1[l][j])
            else:
                genomet.append(genome2[l][j])
        getnome3.append(np.array(genomet))
    return np.array(np.array(getnome3))

In [ ]:
entities = []

populationSize = 100;

#Player Bitd(s)
for i in range(populationSize):
    E1 = Entity()
    E1.addComponenet('bird_y', random.randint(100,300))
    E1.addComponenet('bird_x', 250)
    E1.addComponenet('alive', True)
    E1.addComponenet('score', 0)
    E1.addComponenet('jump', False)
    E1.addComponenet('vel', 0)
    E1.addComponenet('color', 3)
    #AI Part:
    E1.addComponenet('Brain', NeuralNetwork(4,32,1))
    entities.append(E1)

#Pipe(s) 6 of them

P1 = Entity()
P1.addComponenet('pipe_x', 880)
P1.addComponenet('pipe_y', 250)
P1.addComponenet('pipe_speed', 2)
entities.append(P1)

P2 = Entity()
P2.addComponenet('pipe_x', 660)
P2.addComponenet('pipe_y', 250)
P2.addComponenet('pipe_speed', 2)
entities.append(P2)

P3 = Entity()
P3.addComponenet('pipe_x', 440)
P3.addComponenet('pipe_y', 250)
P3.addComponenet('pipe_speed', 2)
entities.append(P3)

systems = []

systems.append(userInputSystem)
systems.append(jumpSystem)
systems.append(physicsSystem)
systems.append(collosionSystem)
systems.append(rewardSystem)
systems.append(AISystem)
systems.append(rendering(660, 600))
systems.append(time(300))
systems.append(resetSysten)

while(True):
    game(entities, systems)

C:\Users\Luka\AppData\Local\Temp\ipykernel_10628\2677707040.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(np.array(getnome3))
C:\Users\Luka\AppData\Local\Temp\ipykernel_10628\2677707040.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(np.array(getnome3))


In [ ]:
np.version.version